In [1]:
import math
import numpy as np
import os
import sys
import time
import argparse
import json
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torch.utils.data as data
import torch.optim as optim
from torch.autograd import Variable
import torch.utils.model_zoo as model_zoo

sys.path.insert(0,'/home/prisimage/tracker/py-MDNet/modules')
from sample_generator import *
from data_prov import *
from model import *
from bbreg import *
from options import *
from gen_config import *

np.random.seed(123)
torch.manual_seed(456)
torch.cuda.manual_seed(789)

In [2]:
seq_home = '/home/prisimage/tracker/py-MDNet/dataset/OTB'
seq_name = 'DragonBaby'
img_dir = os.path.join(seq_home, seq_name, 'img')
gt_path = os.path.join(seq_home, seq_name, 'groundtruth_rect.txt')
img_list = os.listdir(img_dir)
img_list.sort()
img_list = [os.path.join(img_dir,x) for x in img_list]
gt = np.loadtxt(gt_path,delimiter=',')
init_bbox = gt[0]

In [3]:
# Init bbox
target_bbox = np.array(init_bbox)
result = np.zeros((len(img_list),4))
result_bb = np.zeros((len(img_list),4))
result[0] = target_bbox
result_bb[0] = target_bbox

In [4]:
img_list[0]

'/home/prisimage/tracker/py-MDNet/dataset/OTB/DragonBaby/img/0001.jpg'

In [6]:
image = Image.open(img_list[0]).convert('RGB')

In [7]:
# Draw pos/neg samples
pos_examples = gen_samples(SampleGenerator('gaussian', image.size, 0.1, 1.2),
                            target_bbox, opts['n_pos_init'], opts['overlap_pos_init'])

neg_examples = np.concatenate([
                gen_samples(SampleGenerator('uniform', image.size, 1, 2, 1.1), 
                            target_bbox, opts['n_neg_init']//2, opts['overlap_neg_init']),
                gen_samples(SampleGenerator('whole', image.size, 0, 1.2, 1.1),
                            target_bbox, opts['n_neg_init']//2, opts['overlap_neg_init'])])
neg_examples = np.random.permutation(neg_examples)

In [9]:
extractor = RegionExtractor(image, pos_examples, opts['img_size'], opts['padding'], opts['batch_test'])

In [10]:
for i, regions in enumerate(extractor):
    

torch.Size([256, 3, 107, 107])
torch.Size([244, 3, 107, 107])


In [12]:
from utils import *

In [17]:
region = np.zeros((107,107,3),dtype='uint8')
image = np.asarray(image)

In [23]:
for i,sample in enumerate(pos_examples):
    region = crop_image(image,sample,107,16)
    #print(region)
    region.transpose(2,0,1)
    region = (region - 128.)/255.
    img = Image.fromarray(region)
    filename = 'pos_example'+str(i)+'.jpg'
    img.save(filename)

TypeError: Cannot handle this data type

In [3]:
np.array([1,2,3,4])

array([1, 2, 3, 4])